In [1]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import seaborn as sns

In [3]:
payts_datafile = '/mnt/nfs/work1/jtaneja/jmugyenyi/REG_consumption_data/smoothed_payments_06142021.pck'
df = pd.read_pickle(payts_datafile)
df.head()

,meter_serial_number_consumer_id,meter_serial_number,consumer_id,month,year,amounts
0,4226691618_20130515 102321.279 0002030 2194,4226691618,20130515 102321.279 0002030 2194,1,2014,706.166667
1,4226691618_20130515 102321.279 0002030 2194,4226691618,20130515 102321.279 0002030 2194,1,2015,1821.733333
2,4226691618_20130515 102321.279 0002030 2194,4226691618,20130515 102321.279 0002030 2194,1,2016,564.000000
3,4226691618_20130515 102321.279 0002030 2194,4226691618,20130515 102321.279 0002030 2194,1,2017,1150.388889
4,4226691618_20130515 102321.279 0002030 2194,4226691618,20130515 102321.279 0002030 2194,1,2018,1267.500000


In [4]:
len(df)

32669139

In [5]:
consumption_datafile = '/mnt/nfs/work1/jtaneja/jmugyenyi/REG_consumption_data/smoothed_folder/merged_smoothed_monthly_by_meternumber_consumerid_march032021.pck'
df1 = pd.read_pickle(consumption_datafile)
len(df1)

32670278

In [6]:
df['trans_period'] = pd.to_datetime(df[['year','month']].assign(day= lambda x: x['month'].apply(
    lambda y: 31 if y in    [1,3,5,7,8,10,12] else (30 if y in [4,6,9,11] else (28 if y==2 else np.nan)))))
meta_datafile = '/mnt/nfs/work1/jtaneja/jmugyenyi/REG_consumption_data/metadata_folder/REG_metadata_March_3_2021.pck'
meta_df = pd.read_pickle(meta_datafile)
meta_df['meter_serial_number_consumer_id'] = meta_df['meter_serial_number'].astype(str) +'_'+ meta_df['consumer_id']
meta_df['installation_year'] = meta_df.installation_date.dt.year
b = ['10. Residential', '2. T1 Tx FC1 AR STS']
res_meta = meta_df[meta_df.vending_category_name.apply(lambda y: any(x in b for x in y))]

In [ ]:
%%time
## quartiles for low consumption

# lists
quartiles = [0.25, 0.50, 0.75]          # inter-quartile range
lower_quat_cons_list=[]                 # lower quartile kWhs
median_cons_list=[]                     # median kWhs
upper_quat_cons_list=[]                 # upper quartile kWhs
customers_per_conn_year =[]             # number of customers per installation year
customer_per_month_per_conn_year = {}   # number of customers per installation year per transaction month
years_list = [2010,2011,2012,
              2013,2014,2015,2016,2017] # installation years


for year in years_list:
    if year==2010:
        meta_year_filter = res_meta[res_meta.installation_year <= 2010] # filter installation years b4 and equal to 2010
    else:
        meta_year_filter = res_meta[res_meta.installation_year == year] # filter based off installation year
        
    df_by_year = df[df.meter_serial_number_consumer_id.isin(
        meta_year_filter.meter_serial_number_consumer_id)] # create subset dataframe with single year of installation
    
    x = df_by_year.groupby(['meter_serial_number_consumer_id','year']
                          ).mean()['kWhs'].reset_index() # compute average consumption per year
    if year <=2013:
        # average consumption in initial years of connection
        low_consumers = x[(x.year==2013) & (x.kWhs <10)].meter_serial_number_consumer_id.unique().tolist()
    else:  
        # average consumption in initial years of connection
        low_consumers = x[(x.year==year) & (x.kWhs <10)].meter_serial_number_consumer_id.unique().tolist()
        
    # create subset of low consumption customers
    low_df_by_year = df_by_year[df_by_year.meter_serial_number_consumer_id.isin(low_consumers)]
    
    # count number of low consumers per transaction period(monthly)
    customer_per_month_per_conn_year[year] = low_df_by_year.groupby(
        ['trans_period'])['meter_serial_number_consumer_id'].count()
    
    quartile_cons = low_df_by_year.groupby(['trans_period']).quantile([0.25,0.5,0.75])['kWhs']
    temp = quartile_cons.reset_index().rename(columns={'level_1':'quartiles'})
    temp.loc[ (temp.trans_period > '2012-07-01') & (temp.trans_period <= '2015-08-31'), 'tariff'] = 134
    temp.loc[ (temp.trans_period > '2015-08-31') & (temp.trans_period <= '2016-12-31'), 'tariff'] = 182
    temp.loc[ (temp.trans_period > '2016-12-31'), 'tariff'] = 89
    temp['payment'] = temp['kWhs'] * temp['tariff']
    
    
    for quart in quartiles:
        tmp = temp[temp.quartiles==quart]
        tmp = tmp.set_index(['trans_period'])
        tmp = tmp.drop(['quartiles'],axis=1)
        
        if quart==0.25:
            lower_quat_cons_list.append(tmp)
        elif quart==0.50:
            median_cons_list.append(tmp)
        elif quart==0.75:
            upper_quat_cons_list.append(tmp)
    
    cust_count = low_df_by_year.meter_serial_number_consumer_id.nunique()
    if year==2010:
        customers_per_conn_year.append(f"{year} & before : {cust_count} customers")
    else:
        customers_per_conn_year.append(f"{year} : {cust_count} customers")
#     break


In [ ]:
1
